# Homework 4 :


In [62]:
import pandas as pd
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display, Math, Latex


# I - Guidelines

We want to predict a category, i.e. the skin color of the player. Let's figure out what to do following the diagram below, from scikit-learn. 

In [63]:
Image(url= "http://scikit-learn.org/stable/_static/ml_map.png", width=500)

# II - The Data
Let's take a look at the data and already drop what is not useful for the predictions.

In [64]:
df = pd.read_csv('CrowdstormingDataJuly1st.csv')
df.head(3)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [65]:
df.columns

Index(['playerShort', 'player', 'club', 'leagueCountry', 'birthday', 'height',
       'weight', 'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp'],
      dtype='object')

In [66]:
df = df.set_index('playerShort') # set short player number as index

df = df[['height','weight', 'position','games','victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards','rater1', 'rater2','refNum','refCountry', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp']] # considering birthday (age) is not useful because we don't have the dates of individual matches


#df.head()
print(len(df))
df.head(2)

146028


,height,weight,position,games,victories,ties,defeats,goals,yellowCards,yellowReds,...,rater1,rater2,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
playerShort,,,,,,,,,,,,,,,,,,,,,
lucas-wilchez,177.0,72.0,Attacking Midfielder,1,0,0,1,0,0,0,...,0.25,0.50,1,1,0.326391,712.0,0.000564,0.396000,750.0,0.002696
john-utaka,179.0,82.0,Right Winger,1,0,0,1,0,1,0,...,0.75,0.75,2,2,0.203375,40.0,0.010875,-0.204082,49.0,0.061504


In [67]:
df.index.is_unique # but it's not an unique index

False

In [68]:
df.position.unique()  # We have to replace this with numerical values, as we do with the mapping below. 

array(['Attacking Midfielder', 'Right Winger', nan, 'Center Back',
       'Right Midfielder', 'Left Fullback', 'Defensive Midfielder',
       'Goalkeeper', 'Right Fullback', 'Left Winger', 'Left Midfielder',
       'Center Forward', 'Center Midfielder'], dtype=object)

In [69]:
#df.yellowCards.sort_values(ascending=False) # sorting by the number of cards, just to see

The prediction of the skin color is checked with rater1 and rater2. We chose to drop the NaN data in the raters and choose only the data on which they agree. 

Notice that for a Random Forest classifier, there is no need to normalize the features, as no comparison is made between their magnitudes.

In [70]:
df.dropna(how='any', axis=0, inplace=True) # dropping NaN


df2 = df[df.rater1 == df.rater2]
df2['rater1'] = df2['rater1']*4  # the classifier only deals with integers
df2['meanIAT'] = df2['meanIAT']*1000  # the classifier only deals with integers
df2['meanExp'] = df2['meanExp']*100  # the classifier only deals with integers
df2['seIAT'] = df2['seIAT']*10000  # the classifier only deals with integers
df2['seExp'] = df2['seExp']*10000  # the classifier only deals with integers

df2 = np.round(df2,0) # the classifier only deals with integers

df2['position'] = df2['position'].map({'Goalkeeper':0, 'Attacking Midfielder':1, 'Right Winger':2, 'Left Winger':2,
       'Center Back':3, 'Left Fullback':4,'Right Fullback':4, 'Defensive Midfielder':5, 'Left Midfielder':6,
       'Right Midfielder':6,'Center Midfielder':7,'Center Forward':8}) # notice I've used symmetry between R and L


# remember goalkeepers are less likely to get a card. Not taken into account yet

df2.drop('rater2', axis=1, inplace=True)  

df2 = df2[(df2 >= 0)].dropna(how='any') # dropping negative values and NaN

len(df2)

C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

87636

In [71]:
#df2.index.duplicated()  # identifying duplicates: the firsts to appear are not considered duplicates
#df2.rater1.duplicated()

In [72]:
# I DON#T WANT TO ELIMINATE DUPLICATES, THE REASON IT'S REPEATED IS BECAUSE THEY PLAY MORE THAN 1 MATCH
#df2 = df2[df2.index.duplicated()==False] #eliminating duplicates. As the first to appear is not identified, it is kept!
#df2.index.is_unique

We see that we have labeled data, this means we can use a classification algorithm. Moreover, with *less* than 100K sample, we can use [Linear Support Vector Classification](https://en.wikipedia.org/wiki/Support_vector_machine). Now, for the moment, let's suppose it won't work, neither K-Neighbors Classifiers. Let's directly move to Ensemble Classifiers as asked in the HW assignment.

#  III - Machine Learning

Selecting the attributes and passing as X for scikit

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split #deprecated: from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel

y = df2.rater1
df2.drop('rater1', axis=1, inplace=True)  
X = df2

In [74]:
print(y.shape)
X.shape

(87636,)


(87636, 19)

Let's take a look at feature importance with default hyperparameters

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X,y)  #with default hyperparameters

importances = rfc.feature_importances_
sorted_ind = np.argsort(importances)[::-1]
sns.barplot(x=X.columns.values[sorted_ind], y=rfc.feature_importances_[sorted_ind], palette='RdBu_r')
plt.xticks(rotation=45)
plt.title('Features importance')

## Cross-validation

Given that performing feature selection on the whole data before doing cross-validation is likely to bias the prediction, one should do feature selection at each fold of the CV procedure. Moreover, feature importance and hyperparameter importance interact with each other, so one should also perform hyperparameter tuning at each fold of the CV. This is done by selecting features which have a better performance for each set of hyperparmaters. 
Below, we implement this CV-scheme:

In [80]:
import statistics
import numpy as np
import pandas as pd

X_ = X
y_ = y

n_folds = 8  #how many parts the sample is split into train and test sets

#n_trees = list(range(25,100,15))
#tree_depth = list(range(25,85,15))

n_trees = list(range(10,110,20))
tree_depth = list(range(10,60,10))

ma=np.zeros((n_folds,len(tree_depth),len(n_trees)))

X_folds = np.array_split(X_, n_folds)
y_folds = np.array_split(y_, n_folds)

scores_folds = {}
M_scores = {}
A_scores = {}


scores = []
a_scores = []
m_scores = []
bestMscore = []
bestAscore = []

j=0

for l in range(len(n_trees)):
    for m in range(len(tree_depth)):
        for k in range(n_folds):
            X_train = list(X_folds) # We use 'list' to copy, in order to 'pop' later on
            X_test  = X_train.pop(k) # remove the item at the k position in the list, and return it
            X_train = np.concatenate(X_train) # reconstructs the list without item k
            y_train = list(y_folds)
            y_test  = y_train.pop(k)
            y_train = np.concatenate(y_train)
            
            # doing feature selection
            rf = RandomForestClassifier(n_estimators=n_trees[l], max_depth=tree_depth[m])
            rf = rf.fit(X_train, y_train)
            rf.feature_importances_  
            model = SelectFromModel(rf, threshold=0.04, prefit=True)# selecting features with scores above 'threshold'
            X_train = model.transform(X_train)
            X_test = model.transform(X_test)
            
            f_score = rf.fit(X_train, y_train).score(X_test, y_test)
            
            #ma[l,m,k] = f_score
                        
            scores.append(f_score)
            #print ("scores= %s in fold %s" %(scores,k+1))
             
                     
        m_scores = statistics.median(scores)
        a_scores = statistics.mean(scores)
        bestMscore.append(m_scores)
        bestAscore.append(a_scores)
        
        # Dictionaries with the median and the average of the fold scores given a set of hyperparamters: 
        M_scores[n_trees[l],tree_depth[m]]=m_scores 
        A_scores[n_trees[l],tree_depth[m]]=a_scores
                 
        scores = []
            
        #print("averages = %s with %s trees and depth = %s" %(bestAscore,n_trees[l],tree_depth[m]))
        if bestMscore[j] == max(bestMscore):
            best_M = bestMscore[j]
            Mbest_k = k
            Mbest_n = n_trees[l]
            Mbest_d = tree_depth[m]
        if bestAscore[j] == max(bestAscore):
            best_A = bestAscore[j]
            Abest_k = k
            Abest_n = n_trees[l]
            Abest_d = tree_depth[m]
        j = j + 1 
        

print("best score (median)= %s, n_folds = %s,  n_trees = %s, tree_depth = %s" %(best_M, Mbest_k, Mbest_n, Mbest_d)) 
print("best score (average)= %s, n_folds = %s,  n_trees = %s, tree_depth = %s" %(best_A, Abest_k, Abest_n, Abest_d)) 

best score (median)= 0.898630762209, n_folds = 7,  n_trees = 50, tree_depth = 20
best score (average)= 0.896104524318, n_folds = 7,  n_trees = 90, tree_depth = 20


M_scores

In [ ]:
ma

In [551]:
depth = list(range(5,70,5))
n_trees = list(range(5,80,5))
print(depth)

tuples = list(zip(*[list(range(1,n_folds+1)), depth]))
index = pd.MultiIndex.from_tuples(tuples, names=['Depth', 'fold'])
index

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]


MultiIndex(levels=[[1, 2, 3], [5, 10, 15]],
           labels=[[0, 1, 2], [0, 1, 2]],
           names=['Depth', 'fold'])

In [586]:
index

MultiIndex(levels=[[0], [5, 10, 15, 20, 25, 30, 35]],
           labels=[[0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6]],
           names=['fold', 'depth'])

In [585]:
depth = list(range(5,40,5))
n_trees = list(range(5,50,5))
#print(depth)

#tupl = list(zip(*[list(range(1,n_folds+1)), depth]))
tupl = list(zip(*[list(range(1))*len(depth),depth]))
index = pd.MultiIndex.from_tuples(list(zip(*[list(range(1))*len(depth),depth])), names=['fold','depth'])
index2 = pd.MultiIndex.from_tuples(list(zip(*[list(range(1))*len(depth),depth])), names=['fold','depth'])

scores_table = pd.DataFrame(index=index, columns=n_trees)
index

MultiIndex(levels=[[0], [5, 10, 15, 20, 25, 30, 35]],
           labels=[[0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6]],
           names=['fold', 'depth'])

In [597]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]


depth = [5,10,15,20]
#print(dep)


flist= []
flist2= []

for i in range(len(depth)):
    #llist = pd.Series([i]*len(depth),index=None)
    #print (llist)
    ls= np.array([i+1]*len(depth))
    ls= ([i+1]*len(depth))
    flist.append(ls)
    ls2= np.array([depth[i]]*len(depth))
    ls2= depth
    flist2.append(ls2)
    

print(flist)
print(flist2)


llist = np.concatenate([flist,flist2],axis=1)

tupl2 = list(zip(*[llist]))
print(llist)
print(tupl2)
m=np.matrix((5,5))
m

[[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]]
[[5, 10, 15, 20], [5, 10, 15, 20], [5, 10, 15, 20], [5, 10, 15, 20]]
[[ 1  1  1  1  5 10 15 20]
 [ 2  2  2  2  5 10 15 20]
 [ 3  3  3  3  5 10 15 20]
 [ 4  4  4  4  5 10 15 20]]
[(array([ 1,  1,  1,  1,  5, 10, 15, 20]),), (array([ 2,  2,  2,  2,  5, 10, 15, 20]),), (array([ 3,  3,  3,  3,  5, 10, 15, 20]),), (array([ 4,  4,  4,  4,  5, 10, 15, 20]),)]


matrix([[5, 5]])

In [603]:
m=np.zeros((2,3,4))

AttributeError: 'numpy.ndarray' object has no attribute 'to_frame'

In [212]:

df1 = pd.DataFrame(index=n_trees, columns=tree_depth, keys='a')
df2 = pd.DataFrame(index=n_trees, columns=tree_depth)
frames = [df1, df2]
#result = pd.concat(frames, keys=['a','b'])
#result.ix['a'].iloc[0,0]
df1.ix['a']

TypeError: __init__() got an unexpected keyword argument 'keys'

In [535]:
ind = MultiIndex(levels=[[u'1', u'2', u'3'], [u'5', u'10',u'15']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[u'fold', u'depth'])

NameError: name 'MultiIndex' is not defined

In [180]:
df

,5,10
5,2,NaN
10,NaN,NaN
